IMPORTS AND CONFIGURATION

In [ ]:
import numpy as np
import pandas as pd
import os
import importlib
import sys
import copy
import diskcache

In [ ]:
backtesting_path = r'C:\Users\vinayak\Desktop\Backtesting'
print("path before", sys.path)
if backtesting_path not in sys.path:
    print("The backtesting folder was not in the systems path")
    sys.path.append(backtesting_path)
print("path after", sys.path)
print()

path before ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packages\\Pythonwin']
The backtesting folder was not in the systems path
path after ['c:\\Program Files\\Python312\\python312.zip', 'c:\\Program Files\\Python312\\DLLs', 'c:\\Program Files\\Python312\\Lib', 'c:\\Program Files\\Python312', '', 'C:\\Users\\vinayak\\AppData\\Roaming\\Python\\Python312\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32', 'c:\\Program Files\\Python312\\Lib\\site-packages\\win32\\lib', 'c:\\Program Files\\Python312\\Lib\\site-packag

In [ ]:
# from scipy.optimize import minimize
from Modules import Plot
from Modules import Helpers
from Modules import Data as data
from Modules import Utility as util
from Modules import TradeAndLogics as TL
from Modules import Data_Processing as dp
from Modules.enums import Option, LongShort, DB, FNO, Leg, OHLC, PTSL

In [ ]:
pwd()

'c:\\Users\\vinayak\\Desktop\\Backtesting\\Dispersion\\Optimizations\\Month_BNF_ICICI_HDFC'

In [ ]:
original_stdout = util.const(sys.stdout)
original_stdout.value

TRADING/ STRATEGY PARAMETERS

In [ ]:
start_date = '2023-01-01'
end_date = '2024-01-01'

In [ ]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [ ]:
# from datetime import timedelta
# start_date = pd.to_datetime(entry - timedelta(1))
# end_date = pd.to_datetime(exit + timedelta(1))

In [ ]:
buying_delta_threshold_per_lot = 5
selling_delta_threshold_per_lot = 1
notional_vega_buying = 6000 # rs
notional_vega_selling = 10000 # rs
profit_target = 100000
stop_loss = 100000

In [ ]:
index_symbol = 'BANKNIFTY'
expiry_type = 'I'
# expiry_type_next = 'II'
risk_free_rate = 0.1 # (10% interest rate)
timeframe = 15 # mins
look_back_window = 25*4
# Trade/ Strategy Parameters
epsilon = 0.1
moneyness_ironfly = 0
price_factor = 5
ICView = LongShort.Short

In [ ]:
def tell_delta_threshold_per_lot(ticker):
    if ticker.intention == LongShort.Long:
        return buying_delta_threshold_per_lot
    return selling_delta_threshold_per_lot

def tell_notional_vega(ticker):
    if ticker.intention == LongShort.Long:
        return notional_vega_buying
    return notional_vega_selling

In [ ]:
strategy_desc = f"{start_date.strftime('%d%b_%H_%M')}_{end_date.strftime('%d%b_%H_%M')}"
strategy_desc

'01Jan_00_00_01Jan_00_00'

Strategy Specific Data Dealing and Modifications

In [ ]:
index_symbol = 'BANKNIFTY'
index_lot_size = 15

NEAR MONTH (I) DATA FETCHING AND PROCESSING

In [ ]:
import Dispersion.DispersionAdjustedFunctionality as daf

In [ ]:
basket = daf.RawWeightedPortfolio()
basket.insert('HDFCBANK', 550, 27.04)
basket.insert('ICICIBANK', 700, 23.03)
# basket.insert('KOTAKBANK', 400, 11.72)
# basket.insert('SBIN', 750, 11.27)
# basket.insert('AXISBANK', 625, 11.18)

In [ ]:
def get_vega_legs(ticker, timestamp, *Legs):
    vega = 0
    for leg in Legs:
        if leg.Instrument == FNO.FUTURES:
            continue
        greeks = ticker.Greeks(timestamp, leg.Instrument, leg.Strike)
        if greeks is None:
            return None
        vega += greeks['vega'] * leg.Position.value
    return abs(vega)

In [ ]:
def entry_signal(timestamp, trade_count):
    if trade_count != 0:
        return False 
    if timestamp.hour == 9 and timestamp.minute == 30:
        return True
    return False
def exit_signal(timestamp, *portfolio):
    if TL.isLastNdays(timestamp, 7, *portfolio):
        return True
    return False

In [ ]:
def get_lots_for_entry(ticker, timestamp, **kwargs):
    logging_information = {}
    legs = Helpers.get_legs_ironfly_WithFarOptionsOfPrice_ATMpriceXfactor(ticker, timestamp, 1, ticker.intention, price_factor)
    vega_ticker_ironfly = get_vega_legs(ticker, timestamp, *legs)
    if vega_ticker_ironfly is None:
        return None, None
    unweighted_vega = vega_ticker_ironfly * ticker.lot_size
    notional_vega = tell_notional_vega(ticker)
    idea_for_weight = ticker.weight
    logging_information = {
        'Lot Size': ticker.lot_size,
        'Weight': ticker.weight,
        'Vega per IronFly': vega_ticker_ironfly,
        'Vega per Lot IronFly': vega_ticker_ironfly * ticker.lot_size,
        'Target Notional Vega': notional_vega * ticker.weight
    }
    
    # if 'vega_neutral' in kwargs and kwargs['vega_neutral']:
    
    if 'ic_neutral' in kwargs and kwargs['ic_neutral'] == True:
        if ticker.is_component:
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    elif 'theta_neutral' in kwargs and kwargs['theta_neutral'] == True:
        strike_ticker, _ = ticker.find_moneyness_strike(timestamp, 0, Option.Call)
        logging_information['ATM Strike'] = strike_ticker
        if ticker.is_component:
            import math
            idea_for_weight *= math.sqrt(ticker.get_ic_at(timestamp))

    lots_ticker = int(np.round(notional_vega * idea_for_weight/ unweighted_vega))
    logging_information['Lots to take position'] = lots_ticker
    logging_information['Vega Satisfied'] = lots_ticker * unweighted_vega
    cashflow = 0
    for leg in legs:
        token = TL.Token(ticker, leg.Instrument, leg.Strike)
        cashflow += (((token.data[timestamp] * lots_ticker * ticker.lot_size) * leg.Position.value))
    logging_information['CashFlow'] = cashflow
        

    print(f">> {ticker.symbol}")
    for info_key, info_value in logging_information.items():
        print(f"  {info_key}: {info_value}")
    return lots_ticker

In [ ]:
def take_dispersion_position(timestamp, remarks, ticker, lots):
    legs = []
    legs = Helpers.get_legs_ironfly_WithFarOptionsOfPrice_ATMpriceXfactor(ticker, timestamp, 1, ticker.intention, price_factor)
    for leg in legs:
        key = f'{ticker.symbol}_{leg.id()}'
        if key not in ticker.tokens.keys():
            ticker.tokens[key] = TL.Token(ticker, leg.Instrument, leg.Strike, leg.LegName)
        token = ticker.tokens[key]
        token.add_position(timestamp, lots, leg.Position)
    ticker.take_position(timestamp, remarks, *legs)


In [ ]:
class DispersionPTSL(TL.PTSLHandling):
    def __init__(self, profit_target, stop_loss, *tickers):
        super().__init__(profit_target, stop_loss, *tickers)

    def is_valid(self, timestamp):
        # if PTSL is active, update to make sure it's correct
        if self.active_ptsl:
            self.update_validity(timestamp)
        return not self.active_ptsl   
    
    def update_validity(self, timestamp):
        if timestamp.date() != self.triggered_at.date():
            self.reset()
            print(f"New day, PTSL for Intraday Strategy is resetted")

In [ ]:
def squareoff(timestamp, remarks, logging = True):
    print("************  SQUARE OFF BEGINS  *********** ")
    for ticker in portfolio:
        TL.squareoff_ticker(timestamp, remarks, ticker, logging)
    print("* SQUARE OFF TRADES SAVED, TOKENS IN TICKERS UPDATED")
    print("************  SQUARE OFF COMPLETE  *********** ")


In [ ]:
def UpdateDispersionTickers(timestamp, remarks, hedge, logging_token_update, logging_hedging):
    for ticker in portfolio:
        delta_threshold_per_lot = tell_delta_threshold_per_lot(ticker)
        TL.HandleUpdate(ticker, timestamp, remarks, hedge, delta_threshold_per_lot, logging_token_update, logging_hedging)


In [ ]:
def zoom_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    performance = Helpers.zoom_tokens_performance_bar_by_bar(*portfolio, start=start, end=end)
    Plot.save_df_to_excel(performance, file_name)

In [ ]:
def visualise_dispersion_trade(start, end, file_name):
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    result_df = Helpers.get_summary_portfolio(*portfolio, start=start, end=end)
    fig = Plot.plot_df(result_df, *(result_df.columns))
    Plot.save_plot(fig, file_name)

In [ ]:
def save_dispersion_trades(start, end, file_name):
    trades_dict = {}
    trades_dict['All Trades'] = Helpers.get_trades_portfolio(*portfolio, start=start, end=end)
    for ticker in portfolio:
        trades_dict[ticker.symbol] = Helpers.get_trades_ticker(ticker, start=start, end=end)
    Plot.save_df_to_excel(trades_dict, file_name)

In [ ]:
def get_portfolio(start, end, folder):
    path = os.path.join(folder, f"LOGS_Data_and_Initialization.txt")
    file = open(path, 'w', buffering = 1)
    sys.stdout = file

    constituents = {}
    ohlc = OHLC.close
    index = daf.ticker(index_symbol, index_lot_size, True, start, end, expiry_type, True, timeframe, True, 0.1)
    index.initializeDispersion(constituents, False, 1)
    index.set_ohlc(ohlc)
    index.set_intention(ICView)
    for stock in basket.Symbols():
        constituents[stock] = daf.ticker(stock, basket.LotSize(stock), True, start, end, expiry_type, True, timeframe, False, 0.1)
        constituents[stock].initializeDispersion({}, True, basket.Weight(stock))
        constituents[stock].set_ohlc(ohlc)
        constituents[stock].set_intention(ICView.opposite())
    portfolio = [index] + [component for component in index.components.values()]
    
    sys.stdout = original_stdout.value
    file.close()
    return portfolio, index

In [ ]:
a = 2
def f():
    print(a+2)

In [ ]:
f()

4


In [ ]:
sys.stdout = original_stdout.value

In [ ]:
def trading_loop(index, portfolio, timestamps, folder):
    path = os.path.join(folder, f'LOGS_Trading_Logic_for_{strategy_desc}.txt')
    file = open(path, 'w', buffering = 1)
    sys.stdout = file
    for ticker in portfolio:
        ticker.reset_trades()

    TrackPTSL = DispersionPTSL(profit_target, stop_loss, *portfolio)

    current_position = None
    trade_start_date, trade_end_date, trade_count = None, None, 0
    for timestamp in timestamps:
        print()
        print(f"Timestamp: {timestamp}")
        #######################################################################################################################################################
        if TL.check_existing_position(index):
            
            print(f"Existing Position Check: TRUE | Dispersion position: {current_position}")
            
            # 3:20 (Market Close) SQUAREOFF
            if exit_signal(timestamp, *portfolio):
                print("************  MARKET CLOSE  *********** ")
                UpdateDispersionTickers(timestamp, 'NoHedging', False, True, False)
                squareoff(timestamp, f'Last Week Close Square Off', True)
                trade_end_date = timestamp
                trade_count+=1
                print(trade_start_date, trade_end_date)
                zoom_dispersion_trade(trade_start_date, trade_end_date, os.path.join(folder, f"Zoom_Trade.xlsx"))
                visualise_dispersion_trade(trade_start_date, trade_end_date, os.path.join(folder, f"Visualise_Trade.html"))
                continue

            # PROFIT TARGET AND STOP LOSS SQUAREOFF
            if TrackPTSL.is_valid(timestamp) and TrackPTSL.status(timestamp) != PTSL.Valid:
                print("************  TrackPTSL Trigger hit *********** ")
                print(f"{TrackPTSL.nature} Square off at NetPnl of {TrackPTSL.pnl_last_trade}")
                UpdateDispersionTickers(timestamp, 'NoHedging', False, True, False)
                squareoff(timestamp, f'{TrackPTSL.nature} Square Off', True)
                trade_end_date = timestamp
                trade_count+=1
                print(trade_start_date, trade_end_date)
                zoom_dispersion_trade(trade_start_date, trade_end_date, os.path.join(folder, f"Zoom_Trade.xlsx"))
                visualise_dispersion_trade(trade_start_date, trade_end_date, os.path.join(folder, f"Visualise_Trade.html"))
                continue
                
            # IF NO NEED TO SQUAREOFF, HEDGE IF NEEDED
            UpdateDispersionTickers(timestamp, 'Delta Hedging using Synthetic Futures', True, True, True)
            print("========================================================================================================================================================================")
            continue
        #######################################################################################################################################################
        
        
        #######################################################################################################################################################
        if entry_signal(timestamp, trade_count):
            print("************  ENTRY TIME REACHED  ************  ")
            UpdateDispersionTickers(timestamp, "NoHedging", False, False, False)
            if TL.isTodayAnyExpiry(timestamp, *portfolio):
                print("Expiry Day today. Trading on Expiry Day is not Allowed")
                continue

            if not TrackPTSL.is_valid(timestamp):
                print(f"{TrackPTSL.nature} was Triggered at {TrackPTSL.triggered_at.strftime('%H:%M on %d/%b/%Y')} and Z-Score has not yet reverted back to mean (abs(z) is not yet <= {epsilon})")
                continue
            

            for ticker in portfolio:
                lots_ticker = get_lots_for_entry(ticker, timestamp)
                if lots_ticker == 0:
                    print("Cant take Dispersion Position, lots ticker for target notional vega can't be 0")
                    continue
                take_dispersion_position(timestamp, f'{ICView.name} IC', ticker, lots_ticker)
            

            TrackPTSL.fresh_trade(timestamp)
            current_position = ICView
            trade_start_date = timestamp
            print(f"{ICView} IC Trade executed")


            continue
        #######################################################################################################################################################

        UpdateDispersionTickers(timestamp, "NoHedging", False, True, False)
        print("========================================================================================================================================================================")
        
    file.close()
    sys.stdout = original_stdout.value
    trade_count = 0
    print(trade_start_date, trade_end_date)

In [ ]:
date_range = pd.date_range(start=start_date, end=end_date, freq='ME')
for month in date_range:
    start, end = month.replace(day=1).date(), month.date()
    folder_path = r"C:\Users\vinayak\Desktop\Backtesting\Dispersion\Optimizations\Month_BNF_ICICI_HDFC"
    folder = os.path.join(folder_path, f'{start}_{end}')
    if not os.path.exists(folder):
        os.makedirs(folder)
    portfolio, index = get_portfolio(start, end, folder)
    trading_loop(index, portfolio, index.timestamps, folder)
    

In [ ]:
def metrics(fund_blocked):
    info_metrics = open(f'INFO_Metrics_for_{strategy_desc}.txt', 'w')
    sys.stdout = info_metrics

    print(f"TimeFrame Length: 6 months")
    print(f"Fundblocked in rs: {fund_blocked/100}")
    result_df.index = pd.to_datetime(result_df.index)
    result_df['date'] = result_df.index.date  # Create a new column for the date
    daily_profit = result_df.groupby('date')['running_pnl'].agg(lambda x: x.iloc[-1] - x.iloc[0])
    daily_profit.index = pd.to_datetime(result_df['date'].unique())
    
    ret = daily_profit.sum()/fund_blocked * 100
    print(f"Return: {round(ret)}%")
    
    sharpe_value = (daily_profit.mean() - risk_free_rate*fund_blocked/365)/daily_profit.std()
    print(f"Sharpe: {round(sharpe_value)}")
    
    peaks = result_df['running_pnl'].cummax()
    drawdowns = (result_df['running_pnl'] - peaks)/(peaks)
    max_drawdown = drawdowns.min()
    print(f"Drawdown: {round(max_drawdown)}%")
    monthly_data = daily_profit.resample('ME').sum()
    monthly_profit = monthly_data[monthly_data > 0].sum()
    monthly_loss = abs(monthly_data[monthly_data < 0].sum())
    if monthly_loss != 0:
        profit_loss_ratio = monthly_profit / monthly_loss
    else:
        profit_loss_ratio = float('inf')  # Avoid division by zero
    print("Monthly Profit/Loss Ratio:", round(profit_loss_ratio))
    
    windays = (result_df['running_pnl'] > 0).sum()
    lossdays = (result_df['running_pnl'] < 0).sum()
    if (windays + lossdays) > 0:
        win_ratio = windays / (windays + lossdays)
    else:
        win_ratio = 0
    print("Win Ratio:", round(win_ratio))
    
    streak = (result_df['running_pnl'] > 0).astype(int)  # 1 for winning days, 0 otherwise
    losing_streak = (result_df['running_pnl'] < 0).astype(int)  # 1 for losing days, 0 otherwise
    winning_streaks = streak.groupby((streak != streak.shift()).cumsum()).cumsum()
    losing_streaks = losing_streak.groupby((losing_streak != losing_streak.shift()).cumsum()).cumsum()
    print(f"Longest Winning Streak: {winning_streaks.max()}, Longest Losing Streak: {losing_streaks.max()}")

    sys.stdout = original_stdout.value
    info_metrics.close()